<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Feature Selection on the Titanic Dataset

_Authors: Joseph Nelson (SF)_

---

In this lab you will explore a variety of different feature selection methods in sklearn. You will be using the titanic dataset.

You can load the titanic data as follows:

    psql -h dsi.c20gkj5cvu3l.us-east-1.rds.amazonaws.com -p 5432 -U dsi_student titanic
    password: gastudents

Or alternatively load the dataset from the local folder:

    ./datasets/titanic_train.csv
    

## Some useful feature selection resources

---

- Michigan State Overview on [feature selection](http://www.cse.msu.edu/~cse802/Feature_selection.pdf) and (bonus) Texas A&M on [bidrectional feature selection](http://research.cs.tamu.edu/prism/lectures/pr/pr_l11.pdf)
- Sklearn documentation on [feature selection](http://scikit-learn.org/stable/modules/feature_selection.html)
- Side-by-side comparison of [feature selection tactics](http://blog.datadive.net/selecting-good-features-part-iv-stability-selection-rfe-and-everything-side-by-side/)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### 1. Import the data and perform EDA. Engineer any features you think are predictive of survival.

We'll be working with the titanic datasets - go ahead and import it from the dataset folder (or query for it as described above). 

In [2]:
df = pd.read_csv('/Users/Mahendra/desktop/GA/hw/6.1.2_optimization-feature_selection-lab/datasets/titanic_train.csv')

In [3]:
# A:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
df.shape

(891, 12)

In [7]:
for item in df:
    print item
    print df[item].nunique()
    

PassengerId
891
Survived
2
Pclass
3
Name
891
Sex
2
Age
88
SibSp
7
Parch
7
Ticket
681
Fare
248
Cabin
147
Embarked
3


In [8]:
df.Cabin.unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6', 'C23 C25 C27',
       'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33', 'F G73', 'E31',
       'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101', 'F E69', 'D47',
       'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4', 'A32', 'B4',
       'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35', 'C87', 'B77',
       'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19', 'B49', 'D',
       'C22 C26', 'C106', 'C65', 'E36', 'C54', 'B57 B59 B63 B66', 'C7',
       'E34', 'C32', 'B18', 'C124', 'C91', 'E40', 'T', 'C128', 'D37',
       'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44', 'A34', 'C104', 'C111',
       'C92', 'E38', 'D21', 'E12', 'E63', 'A14', 'B37', 'C30', 'D20',
       'B79', 'E25', 'D46', 'B73', 'C95', 'B38', 'B39', 'B22', 'C86',
       'C70', 'A16', 'C101', 'C68', 'A10', 'E68', 'B41', 'A20', 'D19',
       'D50', 'D9', 'A23', 'B50', 'A26', 'D48', 'E58', 'C126', 'B71',
       'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63', 'C62 C64', 'E24',

In [9]:
cabin_letter = df.Cabin.map(lambda x: 'Z' if pd.isnull(x) else x.split()[0][0])
cabin_letter.unique()

array(['Z', 'C', 'E', 'G', 'D', 'A', 'B', 'F', 'T'], dtype=object)

In [12]:
cabin_dummy = pd.get_dummies(cabin_letter, prefix='cabin')
cabin_dummy.head()

,cabin_A,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_T,cabin_Z
0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1


In [13]:
cabin_dummy.drop('cabin_Z', axis=1, inplace=True)

In [15]:
def cabin_numberer(x):
    try:
        return int(x.split()[0][1:])
    except:
        return 0
cabin_num = df.Cabin.map(cabin_numberer)
cabin_num.unique()

array([  0,  85, 123,  46,   6, 103,  56,  23,  78,  33,  30,  52,  28,
        83,  31,   5,  10,  26, 110,  58, 101,  47,  86,   2,  19,   7,
        49,   4,  32,  80,  36,  15,  93,  35,  87,  77,  67,  94, 125,
        99, 118,  22, 106,  65,  54,  57,  34,  18, 124,  91,  40, 128,
        37,  50,  82,  96,  44, 104, 111,  92,  38,  21,  12,  63,  14,
        20,  79,  25,  73,  95,  39,  70,  16,  68,  41,   9,  48, 126,
        71,  51,  62,  24,  90,  45,   8, 121,  11,   3,  17, 102,  69,
        42, 148])

In [16]:
df['cabin_number'] = cabin_num
df = pd.concat([df, cabin_dummy], axis=1)

In [17]:
df.drop('PassengerId', inplace=True, axis=1)
df.drop('Name', inplace=True, axis=1)
df.drop('Ticket', inplace=True, axis=1)
df.drop('Cabin', inplace=True, axis=1)

In [18]:
df.Age.fillna(df.Age.median(), inplace=True)

In [19]:
df = pd.concat([df, pd.get_dummies(df.Embarked)], axis=1)
df.drop('S', inplace=True, axis=1)
df.drop('Embarked', inplace=True, axis=1)

In [20]:
df['Male'] = df.Sex.apply(lambda x: 'female' not in str(x))
df.drop('Sex', inplace=True, axis=1)

In [21]:
df.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,cabin_number,cabin_A,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_T,C,Q,Male
0,0,3,22.0,1,0,7.2500,0,0,0,0,0,0,0,0,0,0,0,True
1,1,1,38.0,1,0,71.2833,85,0,0,1,0,0,0,0,0,1,0,False
2,1,3,26.0,0,0,7.9250,0,0,0,0,0,0,0,0,0,0,0,False
3,1,1,35.0,1,0,53.1000,123,0,0,1,0,0,0,0,0,0,0,False
4,0,3,35.0,0,0,8.0500,0,0,0,0,0,0,0,0,0,0,0,True


### 2. Set up predictor and target matrices

- target should be `Survived`
- predictor matrix will be all other variables

In [23]:
# A:
cols=list(df.columns)
cols.remove('Survived')
y= df['Survived'].values
x=df[cols]

### 3. Feature selection

Let's use the `SelectKBest` method in scikit learn to see which are the top 5 features. Also load the `f_classif` and `chi2` functions which will be our metrics to evaluate what makes a variable the "best".

```python
from sklearn.feature_selection import SelectKBest, f_classif, chi2
```

- What are the top 5 features for `X` using `f_classif`?
- What are the top 5 features for `X` using `chi2`?


> The f-test is explained variance divided by unexplained variance. High numbers will results if our explained variance, what we know is much greater than unexplained, what we dont know. The Chi2 goodness of fit is the sum of the difference squared between observed and expected divided by expected.

In [27]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif

skb_f = SelectKBest(f_classif, k=5)
skb_chi2 = SelectKBest(chi2, k=5)

skb_f.fit(x,y)
skb_chi2.fit(x,y)
kbest = pd.DataFrame([cols, list(skb_f.scores_), list(skb_chi2.scores_)], 
                     index=['feature','f_classif','chi2 score']).T.sort_values('f_classif', ascending=False)
kbest

,feature,f_classif,chi2 score
16,Male,372.406,92.7024
0,Pclass,115.031,30.8737
4,Fare,63.0308,4518.32
5,cabin_number,52.1563,3175.49
7,cabin_B,28.1172,25.8756
14,C,25.896,20.4644
9,cabin_D,20.6632,19.4896
10,cabin_E,19.1792,18.1406
8,cabin_C,11.8417,10.9367
3,Parch,5.96346,10.0975


### 4. Recursive Feature Elimination (RFE)

Sklearn also offers recursive feature elimination as a class named `RFECV`. Use it in combination with a logistic regression model to see what features would be kept with this method.

When instantiating the `RFECV`:
- `step` indicates what percent of features (or number of features if an integer) to remove at each iteration.
- `cv` indicates the number of cross-validation folds to use for evaluating what features are important.

Store the columns in a variable called `rfecv_columns`.

In [29]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
selector = RFECV(lr, step=1, cv=10, verbose=False)
selector = selector.fit(x, y)
print selector.support_
print selector.ranking_

[ True False  True  True False False  True  True  True  True  True  True
  True  True  True  True  True]
[1 2 1 1 4 3 1 1 1 1 1 1 1 1 1 1 1]


In [31]:
rfecv_columns = np.array(cols)[selector.support_]
rfecv_columns

array(['Pclass', 'SibSp', 'Parch', 'cabin_A', 'cabin_B', 'cabin_C',
       'cabin_D', 'cabin_E', 'cabin_F', 'cabin_G', 'cabin_T', 'C', 'Q',
       'Male'], 
      dtype='|S12')

### 5. Feature elimination using the lasso penalty

The L1 penalty is a popular method for feature selection. As the regularization strength increases more features will be removed.

Load the `LogisticRegressionCV` class.

1. Standardize your predictor matrix (required for regularization!)
- Create a logistic regression cross-validator object :
    - Set `penalty='l1'` (Lasso).
    - Set `Cs=100` (search 100 different regularization strengths).
    - Set `solver='liblinear'` (required for the Lasso penalty).
    - Set `cv=10` for 10 cross-validation folds.
- Fit on the target and standardized predictors.
- Sort the logistic regression coefficients by absolute value. Do the top 5 correspond to those selected by the f-score and chi2?


> Answer: Not completely. That could be due to scaling
> Some features are going to perform better based on the model type and If we had not regularized some could be percieved as much more significant than they are.

Choose which ones you would keep and store them in a variable called `lr_columns`

In [7]:
from sklearn.preprocessing import StandardScaler

# A:

In [8]:
# What Does GridSearch think is best?
from sklearn.linear_model import LogisticRegressionCV

# A:

### 6. Compare features sets

Use the optimized logistic regression from the previous question on the features selected from different methods. 
- `kbest_columns`
- `rfecv_columns`
- `lasso_columns`
- `all_columns`

**Questions:**
- Which scores the highest? (use cross_val_score)
- Is the difference significant?


In [9]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [10]:
# A:

### 7. [Bonus] Display the lasso logistic regression coefficients with a barchart.

Start from the most negative on the left.

In [11]:
# A: